In [1]:
from dotenv import load_dotenv
load_dotenv("/home/marshath/play/chainlink/algovate/.env")

Python-dotenv could not parse statement starting at line 4


True

In [8]:
import time
import pickle
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, Markdown

In [10]:
# import logging
# logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
# logger = logging.getLogger(__name__)

In [11]:
from llama_index import Document
from llama_index.node_parser import SimpleNodeParser
from llama_index import GPTVectorStoreIndex, ResponseSynthesizer
from llama_index.data_structs.node import Node, DocumentRelationship

from langchain.chat_models import ChatOpenAI
from llama_index.retrievers import VectorIndexRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.indices.postprocessor import SimilarityPostprocessor
from llama_index import LLMPredictor, GPTVectorStoreIndex, PromptHelper, ServiceContext
from llama_index import GPTListIndex
from llama_index.indices.list.retrievers import ListIndexLLMRetriever
from llama_index.indices.postprocessor import (
    LLMRerank
)

In [14]:
# Load the data
with open('/home/marshath/play/chainlink/algovate/algovate/data/documents.pkl', 'rb') as f:
    cl_techdoc = pickle.load(f)

# convert to llama documents
llama_cl_techdoc = []
for doc in cl_techdoc:
    llama_cl_techdoc.append(Document(text=doc.page_content, extra_info=doc.metadata))

with open('/home/marshath/play/chainlink/algovate/algovate/data/combined_documents.pkl', 'rb') as f:
    cl_combineddoc = pickle.load(f)

# convert to llama documents
llama_cl_combineddoc = []
for doc in cl_combineddoc:
    llama_cl_combineddoc.append(Document(text=doc.page_content, extra_info=doc.metadata))

# load the ground truths
with open("/home/marshath/play/chainlink/algovate/algovate/data/ground_truths.pkl", "rb") as f:
    ground_truths = pickle.load(f)

df = pd.DataFrame(ground_truths)

In [16]:
# convert it to nodes
parser = SimpleNodeParser()
cl_techdoc_nodes = parser.get_nodes_from_documents(llama_cl_techdoc)
cl_combineddoc_nodes = parser.get_nodes_from_documents(llama_cl_combineddoc)

In [24]:
def retrieve_docs(df, query_engine):
    # Add retrieved_docs column if it doesn't exist
    df['retrieved_docs'] = df.get('retrieved_docs', '')

    for index, row in df.iterrows():
        retrieved_docs = query_engine.retrieve(row['question'])

        docs = ""
        for d in retrieved_docs:
            if docs == "":
                docs = d.node.text
            else:
                docs = docs + "\n\n:::NEXT DOC:::\n\n" + d.node.text

        # Store retrieved docs in the DataFrame
        df.at[index, 'retrieved_docs'] = docs

### Base

In [20]:
llm_predictor = LLMPredictor(llm=ChatOpenAI(temperature=0,))

max_input_size = 4096
num_output = 256
max_chunk_overlap = 20

prompt_helper = PromptHelper(max_input_size, num_output, max_chunk_overlap)

service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, prompt_helper=prompt_helper)


##### techdoc only

In [ ]:
index_base_techdoc = GPTVectorStoreIndex.from_documents(
    llama_cl_techdoc, 
    service_context=service_context
)

query_engine_base_techdoc = index_base_techdoc.as_query_engine()

In [25]:
df1 = df.copy()
for i, gt in df1.iterrows():
    try:
        df1.loc[i, "result"] = query_engine_base_techdoc.query(gt.question)
    except Exception as e:
        logger.error(e)
        df1.loc[i, "result"] = "ERROR"

In [26]:
retrieve_docs(df1, query_engine_base_techdoc)

In [28]:
df1.to_csv("/home/marshath/play/chainlink/algovate/test_llama/techdoc_base.csv")

##### combineddocs

In [29]:
index_base_combineddoc = GPTVectorStoreIndex.from_documents(
    llama_cl_combineddoc, 
    service_context=service_context
)

query_engine_base_combineddoc = index_base_combineddoc.as_query_engine()

In [30]:
df2 = df.copy()
for i, gt in df2.iterrows():
    try:
        df2.loc[i, "result"] = query_engine_base_combineddoc.query(gt.question)
    except Exception as e:
        logger.error(e)
        df2.loc[i, "result"] = "ERROR"

In [ ]:
retrieve_docs(df2, query_engine_base_combineddoc)

In [ ]:
df2.to_csv("/home/marshath/play/chainlink/algovate/test_llama/combineddoc_base.csv")

### LLMReranker

In [ ]:
# based on https://medium.com/llamaindex-blog/using-llms-for-retrieval-and-reranking-23cf2d3a14b6
llm_predictor = LLMPredictor(llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo"))
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, chunk_size_limit=512)
reranker = LLMRerank(choice_batch_size=5, top_n=3, service_context=service_context)


##### techdoc

In [ ]:
index_llmreranker_techdoc = GPTVectorStoreIndex.from_documents(llama_cl_techdoc, service_context=service_context)

query_engine_llmreranker_techdoc = index_llmreranker_techdoc.as_query_engine(
    similarity_top_k=10,
    node_postprocessors=[reranker],
)